In [48]:
import pyedflib
import numpy as np
import xmltodict
import json
import mne
import matplotlib
import math
import pathlib
from mne_extras import write_mne_edf

In [3]:
matplotlib.use('Qt5Agg')

In [4]:
#setting up paths for inputs and outputs
input_path = './input_txt/mzk_20210629/'
output_path_edf = './output_edf/'
output_path_fif = './output_fif/'

ch_one = '2021-06-29_082127_RawData_Ch1.txt'
ch_two = '2021-06-29_082127_RawData_Ch2.txt'
ch_three = '2021-06-29_082127_RawData_Ch1.txt'
ch_four = '2021-06-29_082127_RawData_Ch2.txt'

xml = '2021-06-29_082127.xml'
out_name_edf = '2021-06-29_082127.edf'
out_name_fif = '2021-06-29_082127.fif'
out_name_final = 'final_2021-06-29_082127.edf'

ch_one_path = input_path + ch_one
ch_two_path = input_path + ch_two
ch_three_path = input_path + ch_three
ch_four_path = input_path + ch_four
xml_path = input_path + xml
out_file_path_edf = output_path_edf + out_name_edf
out_file_path_fif = output_path_fif + out_name_fif
out_final_edf = output_path_edf + out_name_final

In [5]:
#read from .txt and convert data into numpy array
#each dataset is composed of two channels (.txt) and one information doc (.xml)
raw_one = []
raw_two = []
raw_three = []
raw_four = []

with open(ch_one_path) as f:
    line = f.readline()
    while line:
        raw_one.append(float(line.strip()))
        line = f.readline()
f.close()

with open(ch_two_path) as f:
    line = f.readline()
    while line:
        raw_two.append(float(line.strip()))
        line = f.readline()
f.close()

with open(ch_three_path) as f:
    line = f.readline()
    while line:
        raw_three.append(float(line.strip()))
        line = f.readline()
f.close()

with open(ch_four_path) as f:
    line = f.readline()
    while line:
        raw_four.append(float(line.strip()))
        line = f.readline()
f.close()

signal = [np.array(raw_one, dtype=np.float32), np.array(raw_two, dtype=np.float32), np.array(raw_three, dtype=np.float32), np.array(raw_four, dtype=np.float32)]



In [6]:
#read .xml doc and extract needed info
fileptr = open(xml_path, "r")

xml_content = fileptr.read()

my_ordered_dict = xmltodict.parse(xml_content)
dict = json.loads(json.dumps(my_ordered_dict))

sample_rate = eval(dict['RECORD_INFO']['Record']['SamplesFreq'])

In [7]:
#setting up info needed for .edf generation and write .edf file
headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch3', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch4', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]
with open(out_file_path_edf, 'w') as output:
    print(out_file_path_edf)
    flag = pyedflib.highlevel.write_edf(output.name, signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/2021-06-29_082127.edf
True


In [8]:
#read the newly created .edf using mne
raw=mne.io.read_raw_edf(out_file_path_edf,preload=False)

Extracting EDF parameters from C:\Users\admin\Desktop\work\my_evaluator\output_edf\2021-06-29_082127.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [9]:
raw

<RawEDF | 2021-06-29_082127.edf, 4 x 1306624 (1276.0 s), ~9 kB, data not loaded>

In [10]:
raw.plot()

<MNEBrowseFigure size 1919x786 with 4 Axes>

In [11]:
#create events using mne
#events are equally spaced out for epoch division
new_events = mne.make_fixed_length_events(raw, duration=2.)
event_dict = {'divide':1}

In [12]:
#filter using frequency
#best: do after the rejection, but will reject 89% of data
raw.load_data()
raw.filter(l_freq=0.5, h_freq=30)

Reading 0 ... 1306623  =      0.000 ...  1275.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (6.601 sec)



<RawEDF | 2021-06-29_082127.edf, 4 x 1306624 (1276.0 s), ~39.9 MB, data loaded>

In [13]:
#reject using values of amplitude
#best: do before the filter, but will reject 89% of data
del (dict)
reject_criteria = dict(eeg=400e-6)       # 400 µV

flat_criteria = dict(eeg=1e-6)           # 1 µV

epochs = mne.Epochs(raw,new_events, reject=reject_criteria, flat=flat_criteria,
                    reject_by_annotation=False, preload=True)

Not setting metadata
Not setting metadata
638 matching events found
Setting baseline interval to [-0.2001953125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 638 events and 718 original time points ...
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch3']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', 'ch4']
    Rejecting  epoch based on EEG : ['ch1', 'ch3']
    Rejecting  epoch based on EEG : ['ch1', 'ch2', 'ch3', '

In [14]:
epochs.plot_drop_log()

Channels marked as bad: none


<Figure size 640x480 with 1 Axes>

In [15]:
# epochs.load_data()
# epochs.filter(l_freq=0.5, h_freq=30)

In [31]:
epochs.plot_psd(average=True)

    Using multitaper spectrum estimation with 7 DPSS windows


<MNELineFigure size 1000x350 with 1 Axes>

In [16]:
epochs.plot()

<MNEBrowseFigure size 1919x786 with 4 Axes>

In [17]:
#https://mne.tools/stable/auto_tutorials/preprocessing/50_artifact_correction_ssp.html#tut-artifact-ssp

#cannot do ecg filter because we dont have any ecg channel or meg channel
#ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=0, n_mag=0, n_eeg=4, ch_name='ch1', reject = None)
eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=0, n_mag=0, n_eeg=1, ch_name='ch1', reject = None)

Including 0 SSP projectors from raw file
Running EOG SSP computation
Using EOG channel: ch1
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10240 samples (10.000 sec)

Now detecting blinks and generating corresponding events
Found 89 significant peaks
Number of EOG events detected: 89
Computing projector
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a two-pass forward

In [18]:
projs = eog_projs
#projs = eog_projs + ecg_projs

In [19]:
epochs.add_proj(projs)

1 projection items deactivated


Number of events,556
Events,1: 556
Time range,-0.200 – 0.500 sec
Baseline,-0.200 – 0.000 sec


In [20]:
#apply eog filter
epochs_cleaned = epochs.copy().apply_proj()

Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


In [21]:
epochs[0:100].plot()
epochs_cleaned[0:100].plot()

<MNEBrowseFigure size 1919x786 with 4 Axes>

In [22]:
#save as .fif format
epochs.save(out_file_path_fif, overwrite=True)

Overwriting existing file.


C:\Users\admin\AppData\Local\Temp/ipykernel_4320/2457816835.py:2: RuntimeWarning: This filename (./output_fif/2021-06-29_082127.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_file_path_fif, overwrite=True)


In [23]:
#convert to data frame format in order to save as .edf
df = epochs.to_data_frame()

In [24]:
#save as .edf
out_raw_one = df['ch1']
out_raw_two = df['ch2']
out_raw_three = df['ch3']
out_raw_four = df['ch4']
out_signal = [np.array(out_raw_one), np.array(out_raw_two), np.array(out_raw_three), np.array(out_raw_four)]

with open(out_final_edf, 'w') as output:
    print(out_final_edf)
    flag = pyedflib.highlevel.write_edf(output.name, out_signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/final_2021-06-29_082127.edf
True
Dropped 0 epochs: 
Channels marked as bad: none
Dropped 0 epochs: 
Channels marked as bad: none
Dropped 0 epochs: 
Channels marked as bad: none


In [ ]:
variance = np.var(out_signal)
print('variance:', variance)

stcs = mne.minimum_norm.compute_source_psd_epochs(epochs[:5], fmin=49, fmax=50,
                                 return_generator=True, verbose=True)

In [58]:
epochs.plot_psd(average=True)

    Using multitaper spectrum estimation with 7 DPSS windows


<MNELineFigure size 1000x350 with 1 Axes>

In [52]:
psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs[0], fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_part, frqs_part = mne.time_frequency.psd_multitaper(epochs[0], fmin=45, fmax=55, tmin=None, tmax=None)
part_sum_pds = np.sum(psds_part)
print(part_sum_pds)
ratio = (part_sum_pds) / (total_sum_pds)
print(ratio)

    Using multitaper spectrum estimation with 7 DPSS windows
7.766212502407423e-07
    Using multitaper spectrum estimation with 7 DPSS windows
3.417219427848496e-10
0.00044001106418208404


In [53]:
psds_total, frqs_total = mne.time_frequency.psd_welch(epochs[0], fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_part, frqs_part = mne.time_frequency.psd_welch(epochs[0], fmin=45, fmax=55, tmin=None, tmax=None)
part_sum_pds = np.sum(psds_part)
print(part_sum_pds)
ratio = (part_sum_pds) / (total_sum_pds)
print(ratio)

Effective window size : 0.250 (s)
7.692120343750769e-11
Effective window size : 0.250 (s)
1.1863634956295364e-13
0.0015423101077628896


In [56]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))

psds_total, frqs_total = mne.time_frequency.psd_welch(epochs[0], fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_part, frqs_part = mne.time_frequency.psd_welch(epochs[0], fmin=8, fmax=12, tmin=None, tmax=None)
part_sum_pds = np.sum(psds_part)
print(part_sum_pds)
ratio = (part_sum_pds) / (total_sum_pds)
print(ratio)

Effective window size : 0.250 (s)
7.692120343750769e-11
Effective window size : 0.250 (s)
1.4786830565810364e-11
0.19223347926197593


In [34]:
#evaluate the signal

#formula from paper

#y1: variance score
y1 = 0
#x1: variance
x1 = 0

if x1 < 50:
    y1 = 0.02*np.power(x1,2)
elif x1 >= 50 and x1 < 100:
    y1 = 0.6*x1 + 20
elif x1 >= 100 and x1 < 2000:
    y1 = 100
elif x1 >= 2000 and x1 < 5000:
    y1 = -0.013333*x1 + 126.6
elif x1 >= 5000 and x1 < 10000:
    y1 = 0.006*x1 + 90
else:
    y1 = 15/0.02*np.power((x1-10000),2)
    
#y2: power voltage score
y2 = 0
#x2: signal that are 50Hz (CN) / Total
x2 = 0

if x2 < 0.01:
    y2 = 1
elif x2 >= 0.01 and x2 < 0.1:
    y2 = 1 - 24.691*np.power((x2-0.01),2)
elif x2 >= 0.1 and x2 < 5:
    y2 = 0.8 - 0.00833*np.power((x2-0.1),2)
elif x2 >= 5 and x2 < 10:
    y2 = 0.9 - np.power(0.06,2)
else:
    y2 = 30 / np.power(x2,2)
    
    
#y3: alpha score
y3 = 0
#x3: alpha wave / Total
x3 = 0

if x3 < 0.5:
    y3 = 2.8 * np.power(x3,2)
else:
    y3 = 1.2 * np.power(x3,2) + 2.4*x3 - 0.2
    
print(y1)
print(y2)
print(y3)



0.0
1
0.0


variance: 1473.0319026085767
